# Système Multi-Agents : Analyse et Exploitation de Transcriptions YouTube avec GPT-4o

Ce système illustre comment **plusieurs agents IA** peuvent collaborer pour réaliser une tâche complexe de manière coordonnée : analyser un contenu (texte ou vidéo YouTube) et produire des réponses ou contenus exploitables.

##  Objectifs

* **Agent 1 (Analyse)** : extraire les informations clés d’un texte ou d’une transcription vidéo et produire un livrable structuré.
* **Agent 2 (Réponse/Rédaction)** : exploiter le livrable d’Agent 1 pour répondre à des questions ou générer des contenus prêts à l’emploi (post LinkedIn, résumé, etc.).
* **Orchestrateur** : coordonner le passage de données entre les agents et afficher les résultats intermédiaires et finaux.


##  Outils et bibliothèques

| Outil / Librairie        | Rôle                                                                       |
| ------------------------ | -------------------------------------------------------------------------- |
| `youtube_transcript_api` | Récupération des sous-titres (transcriptions) depuis une URL YouTube       |
| `agents`                 | Création et exécution des agents IA, déclaration des outils                |
| `dotenv`                 | Chargement de la clé OpenAI depuis un fichier `.env` pour plus de sécurité |
| `openai.types.responses` | Gestion des réponses complètes ou en streaming                             |
| `asyncio`                | Gestion asynchrone du dialogue et des appels agents                        |


## Étapes clés

### 1. Chargement des variables d’environnement

* La clé API OpenAI (`OPENAI_API_KEY`) est lue depuis `.env`.
* Sécurise les identifiants en évitant le stockage en clair dans le code.

### 2. Définition de l’outil `fetch_youtube_transcript()`

* Reçoit une URL YouTube.
* Extrait l’ID vidéo.
* Utilise `.fetch()` pour récupérer les sous-titres (priorité au français, sinon anglais).
* Formate chaque ligne avec `[MM:SS]`.
* Accessible aux agents via `@function_tool`.

### 3. Agent 1 — Analyse

* Entrée : texte brut ou transcript.
* Sortie :

  1. Points clés (5–8 éléments).
  2. Ton & intentions (2–3 phrases).
  3. Angles d’approche (2–3 idées).
  4. Résumé exécutif (120–180 mots).

### 4. Agent 2 — Réponse/Rédaction

* Entrée : livrable d’Agent 1 + consigne utilisateur.
* Sortie : réponse factuelle ou contenu prêt à publier.

### 5. Orchestrateur

* Détecte si entrée = URL YouTube ou texte.
* Si URL : tente transcript localement.
* Lance Agent 1, stocke l’analyse.
* Lance Agent 2 avec l’analyse et la consigne.
* Affiche les aperçus.



##  Flux de données

1. **Entrée** : texte ou URL YouTube.
2. **Préparation** : récupération transcript si possible.
3. **Analyse** (Agent 1) → livrable structuré.
4. **Rédaction** (Agent 2) → contenu exploitable.
5. **Affichage** : aperçu clair à l’utilisateur.




In [1]:
# Exécute une fois si nécessaire
!pip install python-dotenv youtube-transcript-api openai-agents openai ipywidgets -q


###  Chargement des bibliothèques essentielles

Dans cette section, nous importons toutes les bibliothèques nécessaires au bon fonctionnement de notre agent intelligent :

- `re` : pour manipuler les expressions régulières (extraction de l'ID vidéo YouTube)
- `asyncio` : pour exécuter l'application de manière asynchrone, sans blocage
- `dotenv` : pour charger automatiquement la clé API OpenAI depuis un fichier `.env` sécurisé
- `youtube_transcript_api` : pour récupérer les sous-titres (transcription) des vidéos YouTube
- `youtube_transcript_api._errors` : pour gérer proprement les erreurs spécifiques à YouTube (ex : vidéo privée, transcription absente, etc.)
- `agents` : module local définissant la structure de l’agent, les outils utilisables, et l’exécution des dialogues
- `openai.types.responses` : pour gérer les réponses `streamées` du modèle (affichage en temps réel mot par mot)




# Importation des packages

In [2]:
import os
import re
import asyncio
from datetime import datetime

# .env (OPENAI_API_KEY)
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# Framework agents (comme dans ton projet)
from agents import Agent, function_tool, Runner
from openai.types.responses import ResponseTextDeltaEvent

# Transcript YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import (
    NoTranscriptFound, TranscriptsDisabled, VideoUnavailable, CouldNotRetrieveTranscript
)

# (Optionnel) permet d'utiliser 'await' au niveau cellule sous Jupyter
import nest_asyncio
nest_asyncio.apply()

/opt/anaconda3/envs/tf2-env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
import logging                                                   # Permet de configurer les niveaux de journalisation du programme
logging.getLogger("httpx").setLevel(logging.WARNING)             # Réduit les messages de log HTTPX à WARNING uniquement (supprime les logs INFO)


In [4]:
from dotenv import load_dotenv                              # Importe la fonction pour charger les variables d’environnement depuis un fichier .env
import os                                                   # Permet d’accéder aux variables d’environnement via os.getenv()

load_dotenv(dotenv_path=".env", override=True)              # Charge explicitement le fichier .env et écrase les variables existantes si nécessaire

api_key = os.getenv("OPENAI_API_KEY")                       # Récupère la clé API OpenAI depuis l’environnement
print("Clé API chargée :", api_key[:8] + "..." if api_key else "Aucune clé détectée")  # Affiche un extrait de la clé ou un message d’erreur


Clé API chargée : sk-proj-...


In [5]:
#

In [6]:
# =========================
# Helper d'extraction (non-streaming)
# =========================
def extract_run_text(res):
    """
    Extrait le texte final pertinent depuis un objet `RunResult` ou équivalent,
    en tenant compte des différentes structures possibles selon la version de la
    bibliothèque `agents`.

    Cette fonction est conçue pour être robuste face aux variations de format
    renvoyées par `Runner.run(...)` ou `Runner.run_streamed(...)`. Elle teste
    plusieurs chemins d'accès au texte produit, en suivant cet ordre :

    1. **Attributs simples** :
       - Recherche dans les attributs courants (`output_text`, `final_output`,
         `text`, `output`) si l'un est une chaîne non vide.

    2. **Messages "OpenAI-style"** :
       - Cherche dans `output_messages` ou `messages` une liste de messages.
       - Parcourt les messages en sens inverse (du plus récent au plus ancien).
       - Si un message contient un champ `content` texte ou une liste de
         segments avec `text`, les concatène.

    3. **Items** :
       - Cherche une liste `items` (ex. sortie détaillée d'un run).
       - Parcourt les éléments en sens inverse.
       - Repère les éléments de type `message_output_item` et en extrait les
         segments texte de `raw_item.content`.

    4. **Fallback** :
       - Si aucune extraction n'a réussi, renvoie `str(res)` comme valeur de
         secours (représentation brute de l'objet).

    Parameters
    ----------
    res : object
        Objet résultat renvoyé par la fonction `Runner.run(...)` ou similaire.
        Peut être un `RunResult`, un dict ou un objet contenant les attributs
        attendus.

    Returns
    -------
    str
        Chaîne extraite contenant le texte final interprétable par l'utilisateur.
        Peut être une chaîne vide si aucune donnée textuelle exploitable n'est trouvée.

    Notes
    -----
    - La fonction est conçue pour tolérer les changements de structure internes
      entre différentes versions de la lib `agents`.
    - Si la sortie est complexe (par ex. avec plusieurs parties), elles sont
      concaténées avec des sauts de ligne.
    """
    # Tentatives directes
    for attr in ("output_text", "final_output", "text", "output"):
        val = getattr(res, attr, None)
        if isinstance(val, str) and val.strip():
            return val

    # Messages (OpenAI-style)
    msgs = getattr(res, "output_messages", None) or getattr(res, "messages", None)
    if isinstance(msgs, list) and msgs:
        for m in reversed(msgs):
            content = m.get("content") if isinstance(m, dict) else getattr(m, "content", None)
            if isinstance(content, str) and content.strip():
                return content
            if isinstance(content, list):
                parts = []
                for p in content:
                    t = p.get("text") if isinstance(p, dict) else getattr(p, "text", None)
                    if t:
                        parts.append(t)
                if parts:
                    return "\n".join(parts)

    # Items (message_output_item)
    items = getattr(res, "items", None)
    if isinstance(items, list) and items:
        for it in reversed(items):
            it_type = it.get("type") if isinstance(it, dict) else getattr(it, "type", None)
            if it_type == "message_output_item":
                raw = it.get("raw_item") if isinstance(it, dict) else getattr(it, "raw_item", None)
                content = raw.get("content") if isinstance(raw, dict) else getattr(raw, "content", None)
                if isinstance(content, list):
                    parts = []
                    for p in content:
                        t = p.get("text") if isinstance(p, dict) else getattr(p, "text", None)
                        if t:
                            parts.append(t)
                    if parts:
                        return "\n".join(parts)

    # Fallback
    return str(res)


### Récupération et formatage de la transcription

Cette fonction permet d’extraire automatiquement la transcription d’une vidéo YouTube à partir de son URL.  
Elle identifie l’ID de la vidéo, interroge l’API via la méthode `.fetch()` et formate le texte avec des horodatages `[MM:SS]`.  
Elle prend également en compte les cas d’erreur courants (vidéo privée, transcription désactivée, etc.).  



In [7]:
# =========================
# Transcript (impl Python robuste + outil exposé)
# =========================
def fetch_youtube_transcript_impl(url: str) -> str:
    """Récupère la transcription d'une vidéo YouTube et la formate [MM:SS] texte.
       Compatible dicts ET objets FetchedTranscriptSnippet.
    """
    video_id_pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
    match = re.search(video_id_pattern, url or "")
    if not match:
        return "⚠️ URL YouTube invalide."

    vid = match.group(1)
    try:
        ytt_api = YouTubeTranscriptApi()
        transcript_data = None

        # Priorité fr → en → défaut
        try:
            transcript_data = ytt_api.fetch(vid, languages=['fr'])
        except NoTranscriptFound:
            try:
                transcript_data = ytt_api.fetch(vid, languages=['en'])
            except NoTranscriptFound:
                transcript_data = ytt_api.fetch(vid)

        if not transcript_data:
            return "❌ Aucune donnée de transcription récupérée"

        lines = []
        for entry in transcript_data:
            try:
                # Cas objet (FetchedTranscriptSnippet)
                if hasattr(entry, "start") and hasattr(entry, "text"):
                    start = float(entry.start)
                    text = entry.text
                # Cas dict (ancien format)
                elif isinstance(entry, dict) and "start" in entry and "text" in entry:
                    start = float(entry["start"])
                    text = entry["text"]
                else:
                    # Fallback ultra-robuste
                    start = float(getattr(entry, "start", 0.0))
                    text = str(getattr(entry, "text", ""))

                text = (text or "").strip()
                if not text:
                    continue

                m, s = int(start // 60), int(start % 60)
                lines.append(f"[{m:02d}:{s:02d}] {text}")
            except Exception:
                continue

        if not lines:
            return "❌ Transcription vide"

        return "\n".join(lines)

    except TranscriptsDisabled:
        return "❌ Les transcriptions sont désactivées pour cette vidéo."
    except VideoUnavailable:
        return "❌ Vidéo non disponible."
    except CouldNotRetrieveTranscript:
        return "❌ Impossible de récupérer la transcription."
    except Exception as e:
        return f"❌ Erreur : {str(e)}"

# Exposer l’outil pour l’agent (function calling)
from agents import function_tool
fetch_youtube_transcript = function_tool(fetch_youtube_transcript_impl)


##  Agents IA (GPT-5) — Instructions hybrides

### Agent 1 — Analyse
- **Rôle** : Analyse un texte ou une URL YouTube.
- **Si URL** : utilise `fetch_youtube_transcript` pour récupérer le transcript.
- **Sortie** :  
  1. Points clés (5–8)  
  2. Ton & intentions (2–3 phrases)  
  3. Angles d’email (2–3 lignes)  
  4. Résumé exécutif (120–180 mots)  
- **Modèle** : `gpt-5`

### Agent 2 — Réponse & Rédaction
- **Rôle** : Utilise le transcript et l’analyse pour :
  - Répondre à des questions factuelles
  - Générer un post LinkedIn (800–1200 car.) ou Instagram (300–600 car.)
- **Outils** : aucun  
- **Modèle** : `gpt-5`

**Flux** : Utilisateur → Agent 1 (analyse) → Agent 2 (réponse ou rédaction)


In [8]:
# =========================
#  Agents (GPT-5) avec instructions hybrides
# =========================
# Agent 1 — Analyse (outil en secours si URL sans transcript)
AGENT1_INSTRUCTIONS = (
    "Tu es un analyste éditorial. "
    "Tu peux recevoir soit un TEXTE directement, soit une URL YouTube. "
    "Si on te donne une URL YouTube et qu'aucun transcript exploitable n'est fourni dans le message, "
    "alors appelle l'outil fetch_youtube_transcript(url) pour récupérer le transcript AVANT d'analyser. "
    "Ensuite, retourne STRICTEMENT les blocs suivants, en français :\n"
    "1) Points clés (5–8, liste numérotée)\n"
    "2) Ton & intentions de l’auteur (2–3 phrases)\n"
    "3) Angles d’email possibles (2–3, une ligne chacun)\n"
    "4) Résumé exécutif (120–180 mots)\n"
    "Évite les généralités, ancre-toi dans le contenu. Ne rajoute pas d’autre texte."
)

def create_agent_analyse():
    return Agent(
        name="Agent Analyse",
        instructions=AGENT1_INSTRUCTIONS,
        tools=[fetch_youtube_transcript],  # ✅ outil dispo en backup
        model="gpt-5",
    )

# Agent 2 — Réponse & Rédaction
AGENT2_INSTRUCTIONS = (
    "Tu es un assistant de réponses et de rédaction. On te donne :\n"
    "- le transcript ou le texte source (si disponible)\n"
    "- le livrable de l’Agent Analyse (points clés, ton, angles, résumé)\n"
    "Tu dois :\n"
    "1) Répondre aux questions factuelles de l’utilisateur en t’appuyant sur le contenu, OU\n"
    "2) Générer des contenus courts (post LinkedIn 800–1200 car., post Instagram 300–600 car.).\n"
    "Reste concret et fidèle au contenu."
)

def create_agent_responder():
    return Agent(
        name="Agent Réponse & Rédaction",
        instructions=AGENT2_INSTRUCTIONS,
        tools=[],  # pas d’outil ici
        model="gpt-5",
    )


### Lancement de la boucle de dialogue avec l'utilisateur

La fonction `main()` initialise et gère l’interaction en continu entre l'utilisateur et l'agent.

#### Fonctionnement :
- Affiche un message d’accueil et attend une entrée utilisateur
- Gère les commandes de sortie (`exit`, `quit`, etc.)
- Stocke les échanges dans une liste `input_items` pour maintenir le contexte
- Limite l’historique à 8 messages pour éviter les dépassements de capacité du modèle (token limit)
- Transmet les échanges à l’agent via `Runner.run_streamed(...)` pour obtenir une réponse en streaming
- Gère les différents types d’événements retournés : texte généré, appel d’outil, résultats d’outil
- Affiche la réponse en temps réel ligne par ligne

Cette boucle permet de simuler une véritable conversation, tout en exploitant les capacités du modèle GPT-4o et de l’outil `fetch_youtube_transcript` si nécessaire.


# Orchestrateur :
 - SessionState : stocke transcript, analyse, type de source
 - prepare_source : récupère transcript côté Python si possible, sinon garde texte brut
 - run_analysis_pipeline : lance Agent 1 (analyse) avec transcript ou URL
 - ask_with_context : lance Agent 2 (réponse/rédaction) avec transcript + analyse + demande


In [9]:
# =========================
#  Orchestration automatique (analyse + livrables)
# =========================

import json
import textwrap

def force_json_prompt():
    return (
        "Tu dois produire STRICTEMENT un JSON compact (une seule ligne), sans commentaire, sans texte avant/après.\n"
        "Schéma attendu:\n"
        "{\n"
        '  "points_cles": ["", "", "", "", ""],\n'
        '  "post_linkedin": ""\n'
        "}\n"
        "- points_cles : exactement 5 éléments, concis, factuels, fidèles au contenu d’Agent 1.\n"
        "- post_linkedin : ~900 caractères, français, structuré (accroche, développement, conclusion/CTA), lisible, sans hashtags superflus.\n"
        "Ne renvoie surtout pas de Markdown ni de texte hors JSON."
    )

async def run_full_pipeline(text_or_url: str):
    # 1) Analyse (Agent 1)
    analysis = await run_analysis_pipeline(text_or_url)
    if not analysis or analysis.startswith("❌"):
        return {
            "status": "error",
            "error": analysis or "Analyse vide.",
            "points_cles": [],
            "post_linkedin": ""
        }

    # 2) Agent 2 — consigne stricte pour renvoyer un JSON (points_cles + post_linkedin)
    strict_json_instr = force_json_prompt()
    user_task = (
        "À partir du transcript/texte et de l'analyse ci-dessus, fais ceci :\n"
        "1) Donne-moi un résumé clair en **5 points** (courts, factuels, sans jargon inutile).\n"
        "2) Propose un **post LinkedIn** autour de 900 caractères, structuré (accroche → idée centrale → appel à l’action), ton pro, sans emojis excessifs.\n\n"
        "Respecte le format JSON imposé ci-dessous."
    )
    full_context = (
        "=== TRANSCRIPT / TEXTE ===\n" + (state.transcript[:12000] or "") + "\n\n"
        "=== ANALYSE (Agent 1) ===\n" + (analysis[:12000] or "") + "\n\n"
        "=== DEMANDE UTILISATEUR ===\n" + user_task + "\n\n"
        "=== FORMAT ===\n" + strict_json_instr
    )

    res = await Runner.run(
        create_agent_responder(),
        input=[{"role": "user", "content": full_context}]
    )
    raw = extract_run_text(res).strip()

    # 3) Parsing JSON robuste (tolérance aux entêtes/markdown)
    def try_extract_json(s: str):
        # essaie de trouver le premier { ... } équilibré
        start = s.find("{")
        end = s.rfind("}")
        if start != -1 and end != -1 and end > start:
            candidate = s[start:end+1].strip()
            try:
                return json.loads(candidate)
            except json.JSONDecodeError:
                pass
        # dernier recours : tenter directement
        try:
            return json.loads(s)
        except Exception:
            return None

    payload = try_extract_json(raw)
    if not payload or not isinstance(payload, dict):
        return {
            "status": "error",
            "error": "Sortie Agent 2 non parsable en JSON.",
            "raw": raw,
            "points_cles": [],
            "post_linkedin": ""
        }

    points = payload.get("points_cles") or []
    post = payload.get("post_linkedin") or ""
    if not isinstance(points, list):
        points = []
    if not isinstance(post, str):
        post = ""

    return {
        "status": "ok",
        "analysis_excerpt": analysis[:1200],
        "points_cles": points[:5],  # on limite à 5 si plus
        "post_linkedin": post.strip()
    }


In [10]:
async def run_analysis_pipeline(text_or_url: str) -> str:
    """
    Lance l'analyse avec Agent 1.
    - Si transcript Python disponible, l'utilise directement
    - Sinon, passe l'URL brute pour que l'agent appelle l'outil
    - Retourne les 4 blocs attendus et les stocke dans state.analysis
    """
    user_msg = (text_or_url or "").strip()
    if not user_msg:
        return "❌ Aucun contenu."

    # Prépare la source : transcript ou texte brut
    source_text = prepare_source(user_msg)
    a1 = create_agent_analyse()

    # Cas A : transcript récupéré côté Python
    if source_text and not source_text.startswith(("❌", "⚠️")):
        prompt = f"Analyse ce contenu et renvoie les 4 blocs demandés :\n\n{source_text}"
        items = [{"role": "user", "content": prompt}]
    else:
        # Cas B : aucun transcript → l'agent devra appeler l’outil
        items = [{"role": "user", "content": user_msg}]

    # Exécution (non-streaming)
    res = await Runner.run(a1, input=items)
    analysis = extract_run_text(res).strip()

    # Stockage
    state.analysis = analysis if analysis else None
    return analysis if analysis else "❌ Analyse vide."


async def ask_with_context(user_question_or_task: str) -> str:
    """
    Lance Agent 2 pour répondre ou rédiger à partir :
    - du transcript ou texte brut
    - de l'analyse produite par Agent 1
    - de la question / consigne utilisateur
    """
    if not state.transcript:
        return "❌ Aucun transcript/texte source disponible. Lance d’abord l’analyse."
    if not state.analysis:
        return "❌ Aucune analyse disponible. Lance d’abord l’analyse."

    a2 = create_agent_responder()

    # Construit le contexte complet
    context = (
        "=== TRANSCRIPT / TEXTE ===\n" + (state.transcript[:12000] or "") + "\n\n"
        "=== ANALYSE (Agent 1) ===\n" + (state.analysis[:12000] or "") + "\n\n"
        "=== DEMANDE UTILISATEUR ===\n" + (user_question_or_task or "")
    )

    # Exécution (non-streaming)
    res = await Runner.run(a2, input=[{"role": "user", "content": context}])
    return extract_run_text(res).strip() or "❌ Réponse vide."


class SessionState:
    def __init__(self):
        self.transcript = None   # texte transcript/texte brut retenu
        self.analysis = None     # sortie Agent 1
        self.source_kind = None  # "url" ou "text"

state = SessionState()


def is_youtube_url(s: str) -> bool:
    """Détecte si une chaîne est une URL YouTube."""
    return bool(s) and ("youtu.be/" in s or "youtube.com/watch" in s)


def prepare_source(user_input: str) -> str:
    """
    Prépare la source pour l'analyse.
    - Si URL YouTube → tente de récupérer transcript côté Python
    - Sinon → texte brut
    """
    user_input = (user_input or "").strip()
    if not user_input:
        return ""

    if is_youtube_url(user_input):
        state.source_kind = "url"
        # Ici, on appelle ton implémentation Python directe de transcript
        txt = fetch_youtube_transcript_impl(user_input)  
        state.transcript = txt
        return txt

    state.source_kind = "text"
    state.transcript = user_input
    return user_input



In [11]:
# =========================
# Démo rapide
# =========================
SOURCE_INPUT = "https://www.youtube.com/watch?v=vOmo-Q7RrRc&t=643s"  # remplace par une URL transcriptible ou colle un texte

analysis = await run_analysis_pipeline(SOURCE_INPUT)
print("=== ANALYSE (Agent 1) — extrait ===")
print((analysis or "")[:1200])

QUESTION_OR_TASK = "Donne-moi un résumé clair en 5 points, puis propose un post LinkedIn (~900 caractères)."
answer = await ask_with_context(QUESTION_OR_TASK)
print("\n=== RÉPONSE (Agent 2) — extrait ===")
print((answer or "")[:1600])


=== ANALYSE (Agent 1) — extrait ===
1) Points clés
1. Trois tests concrets sur un dataset de churn: analyse exploratoire, modélisation ML, et création d’une app Shiny “end-to-end”.
2. Test 1 (analyse): GPT‑5 a importé la data (pandas), fait des stats descriptives, tenté un modèle baseline, identifié des drivers (géographie, activité, âge, genre) et fourni un rapport Excel; mais peu de visualisations et confusion initiale sur la cible; note: 5/10. Claude 4.1: livrable visuellement séduisant mais analyses manquantes.
3. Test 2 (modélisation): GPT‑5 a livré un pipeline Python (OneHotEncoder, StandardScaler, LogisticRegression), split stratifié, score ~83,7% en test, export du modèle en .joblib + README de déploiement, et propose d’essayer d’autres algos; note: 5/10 (rapport et justifications insuffisants). Claude: surtout de l’HTML/design et une liste d’algos, pas d’exécution; 1/10 “pour la beauté”.
4. Test 3 (app Shiny R): structure complète (import, EDA, prétraitement, modélisation, éva

In [ ]:
# =========================
# 💬 Cellule 8 — Mini chat (AUTO orchestration)
# =========================
import textwrap

DEBUG = True
MAX_PREVIEW = 1500

def _looks_like_url(s: str) -> bool:
    s = (s or "").strip().lower()
    return s.startswith("http://") or s.startswith("https://")

def _is_youtube_url(s: str) -> bool:
    s = (s or "").strip().lower()
    return "youtu.be/" in s or "youtube.com/watch" in s

def _looks_like_long_text(s: str) -> bool:
    # Heuristique simple pour déclencher l'analyse sur un texte “long”
    return len((s or "").strip()) >= 200

async def chat_loop():
    print("=== Multi-Agents YouTube — GPT-5 ===")
    print("Collez une URL YouTube (ou un texte long) → je lance l’analyse + points clés + post LinkedIn automatiquement.")
    print("Ensuite, tapez une courte question/consigne → je réponds avec l’Agent 2.")
    print("Tapez 'exit' pour quitter.")

    while True:
        try:
            user = await asyncio.to_thread(input, "\nYou: ")
        except (EOFError, KeyboardInterrupt):
            print("\nBye.")
            break

        if not user:
            continue
        cmd_raw = user.strip()
        cmd = cmd_raw.lower()

        # Quitter
        if cmd in ("exit", "quit", "bye"):
            print("👋 À bientôt !")
            break

        # 1) Si c'est une URL (YouTube ou autre) OU un texte suffisamment long → orchestration complète
        if _looks_like_url(cmd_raw) or _is_youtube_url(cmd_raw) or _looks_like_long_text(cmd_raw):
            # Orchestration automatique (Agent 1 → Agent 2)
            result = await run_full_pipeline(cmd_raw)
            if result.get("status") != "ok":
                print("❌ Pipeline échoué :", result.get("error", "Erreur inconnue"))
                if result.get("raw"):
                    print("\n--- Sortie brute Agent 2 (debug) ---\n")
                    print(result["raw"][:2000])
            else:
                print("\n=== Agent 1 — Analyse (extrait) ===")
                print(result["analysis_excerpt"])

                print("\n=== Agent 2 — Résultat 1/2 : Points clés ===")
                if result["points_cles"]:
                    for i, p in enumerate(result["points_cles"], 1):
                        print(f"{i}. {p}")
                else:
                    print("Aucun point clé parsé.")

                print("\n=== Agent 2 — Résultat 2/2 : Post LinkedIn (~900 car.) ===")
                if result["post_linkedin"]:
                    print(textwrap.fill(result["post_linkedin"], width=100))
                else:
                    print("Post LinkedIn vide ou non parsé.")
            continue

        # 2) Sinon : courte question/consigne → Agent 2 (si on a déjà une analyse)
        if state.transcript and state.analysis:
            resp = await ask_with_context(cmd_raw)
            if resp.startswith("❌"):
                print("\n[Agent 2 — ERREUR]")
                print(resp)
            else:
                print("\n[Agent 2 — RÉPONSE] (aperçu)")
                print(resp[:MAX_PREVIEW] + ("\n…[tronqué]" if len(resp) > MAX_PREVIEW else ""))
        else:
            print("\nℹ️ Collez d’abord une URL YouTube (ou un texte long ≥ 200 caractères).")
            print("Je lancerai automatiquement l’analyse + les points clés + le post LinkedIn.")

# Lance la boucle :
await chat_loop()


=== Multi-Agents YouTube — GPT-4o (non-streaming) ===
Collez une URL YouTube (ou un texte long) → je lance l’analyse + points clés + post LinkedIn automatiquement.
Ensuite, tapez une courte question/consigne → je réponds avec l’Agent 2.
Tapez 'exit' pour quitter.



You:  https://www.youtube.com/watch?v=7-FFFjlLwos


In [ ]:
pip freeze > requirements.txt


In [ ]:
pip install pipreqs
